# Objectif
Ce fichier a pour objectif de préparer les données pour **la reconnaissance du locuteur (Partie II)** en utilisant les caractéristiques MFCC extraites et réorganisées à l'aide du fichier **Part2_R-Locuteur_MFCC-Extraction.ipynb**. Le processus commence par le chargement des données depuis le dossier "**Features_MFCC_Locuteur**", où elles sont organisées selon la langue, le genre du locuteur, l'identifiant du locuteur, et le type (entraînement/test).

Par la suite, des modèles de Mélanges de Gaussiennes (GMM) seront générés et entraînés pour chaque locuteur à partir de ces caractéristiques. Enfin, ces modèles seront sauvegardés pour une utilisation ultérieure,

In [5]:
# !pip install python_speech_features

In [6]:
import numpy as np
import os
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from sklearn.mixture import GaussianMixture
from datetime import datetime
from joblib import dump
from joblib import load

from google.colab import drive
drive.mount('/content/drive') # Monter (Mount) Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- Chargement des données MFCC pour Locuteur sauvegardées

In [7]:
# Chemin vers le dossier MFCC de Locuteur dans Google Drive
mfcc_locuteur_dir = '/content/drive/My Drive/Reconnaissance/Features_MFCC_Locuteur'

langues = os.listdir(mfcc_locuteur_dir)
print(langues)

['French', 'arabic', 'Japanese', 'Spanish', 'English']


- Charge les caractéristiques MFCC pour un locuteur spécifique à partir des données d'entraînement.

In [8]:
def charger_caracteristiques_mfcc_train_par_locuteur(repertoire_mfcc, langue_cible, genre_cible, locuteur_cible):
    # Liste vide pour stocker les caractéristiques MFCC
    donnees_mfcc_entraine = []

    # Chemin vers le dossier contenant les fichiers MFCC pour l'ensemble train
    dossier_train = os.path.join(repertoire_mfcc, langue_cible, genre_cible, locuteur_cible, 'train')

    # Parcours des fichiers dans le dossier d'entraînement
    for fichier_mfcc in os.listdir(dossier_train):
        # Chargement des caractéristiques MFCC à partir d'un fichier texte
        caracteristiques_mfcc = np.loadtxt(os.path.join(dossier_train, fichier_mfcc), delimiter=',')

        # Ajout des caractéristiques dans la liste des données d'entraînement
        donnees_mfcc_entraine.append(caracteristiques_mfcc)

    # Retourne les données empilées sous forme de tableau numpy
    return np.vstack(donnees_mfcc_entraine)


- Entraîne un modèle GMM avec les données d'entraînement et enregistre le modèle.

In [9]:
def entrainer_modele_GMM_par_locuteur(donnees_entraînement, nombre_composants, langue, genre, id_locuteur, dossier_sortie):
    # Affiche l'heure de début de l'entraînement
    heure_debut = datetime.now().strftime("%H:%M:%S")
    print(f'Début de l\'entraînement à : {heure_debut}')

    # Création et entraînement du modèle GMM
    modele_gmm = GaussianMixture(n_components=nombre_composants, covariance_type='full', max_iter=200, random_state=0)
    modele_gmm.fit(donnees_entraînement)

    # Affiche l'heure de fin de l'entraînement
    heure_fin = datetime.now().strftime("%H:%M:%S")
    print(f'Fin de l\'entraînement à : {heure_fin}')

    # Construction du nom du fichier avec les informations spécifiées
    nom_fichier_modele = f"GMM_{nombre_composants}_{langue}_{genre}_{id_locuteur}_Locuteur.joblib"

    # Construction du chemin complet pour sauvegarder le modèle
    chemin_complet_fichier = f"{dossier_sortie}/{nom_fichier_modele}"

    # Sauvegarde du modèle GMM
    dump(modele_gmm, chemin_complet_fichier)
    # print(f"Modèle sauvegardé sous le nom : {chemin_complet_fichier}\n")

    # Retourne le modèle GMM entraîné
    return modele_gmm

- Ce code effectue l'entraînement de modèles GMM pour des données audio structurées par langue, genre et locuteur.  
- Il crée une organisation hiérarchique dans un répertoire Google Drive, où les modèles GMM sont stockés dans des sous-dossiers dédiés.  
- Les étapes principales incluent :  
1. Initialisation du dictionnaire `modeles_GMM` pour conserver les modèles GMM pour chaque combinaison langue-genre-locuteur.  
2. Création des dossiers dans Google Drive pour chaque langue, genre et locuteur, afin de sauvegarder les modèles de manière organisée.  
3. Chargement des données MFCC pour chaque locuteur depuis un dossier source.  
4. Entraînement de modèles GMM avec différents nombres de composantes (par exemple : 8, 16, 32, etc.).  
5. Sauvegarde des modèles GMM entraînés dans les dossiers correspondants pour un usage futur.  


In [ ]:
### N'exécutez pas cette cellule si les résultats sont déjà calculés et enregistrés dans Google Drive, car ce processus est long.

# Chemin vers le dossier pour stocker les modèles GMM dans Google Drive
dossier_GMMs_Locuteur = '/content/drive/My Drive/Reconnaissance/Models_GMM/R-Locuteur'
os.makedirs(dossier_GMMs_Locuteur, exist_ok=True)

# Initialisation du dictionnaire pour stocker les modèles GMM
modeles_GMM = {}

# Parcours des langues dans le dossier MFCC
langues = os.listdir(mfcc_locuteur_dir)
# langues = ['English']
print(f"Langues détectées : {langues}")

# Parcours des langues
for langue in langues:
    modeles_GMM[langue] = {}

    # Création du répertoire pour la langue dans Google Drive
    dossier_langue = os.path.join(dossier_GMMs_Locuteur, langue)
    os.makedirs(dossier_langue, exist_ok=True)

    print(f"\nDébut d'entraînement pour la langue : {langue}")

    # Parcours des genres (homme/femme) pour chaque langue
    for genre in os.listdir(os.path.join(mfcc_locuteur_dir, langue)):
        modeles_GMM[langue][genre] = {}

        # Création du répertoire pour le genre
        dossier_genre = os.path.join(dossier_langue, genre)
        os.makedirs(dossier_genre, exist_ok=True)

        print(f"Traitement du genre : {genre}")

        # Parcours des locuteurs pour chaque genre
        for locuteur in os.listdir(os.path.join(mfcc_locuteur_dir, langue, genre)):
            modeles_GMM[langue][genre][locuteur] = {}

            # Création du répertoire pour le locuteur
            dossier_locuteur = os.path.join(dossier_genre, locuteur)
            os.makedirs(dossier_locuteur, exist_ok=True)

            print(f"Locuteur : {locuteur}")

            # Chargement des données MFCC d'entraînement pour le locuteur
            donnees_train = charger_caracteristiques_mfcc_train_par_locuteur(mfcc_locuteur_dir, langue, genre, locuteur)

            # Liste des nombres de composants à tester
            liste_composants = [8, 16, 32, 64, 128]
            for nombre_composants in liste_composants:
                print(f"========= Entraînement du GMM avec {nombre_composants} composants ==========")

                # Entraînement du modèle GMM et sauvegarde
                modele_gmm = entrainer_modele_GMM_par_locuteur(donnees_train, nombre_composants, langue, genre, locuteur, dossier_locuteur)

                # Sauvegarde du modèle GMM dans le dictionnaire
                modeles_GMM[langue][genre][locuteur][nombre_composants] = modele_gmm


Langues détectées : ['English']

Début d'entraînement pour la langue : English
Traitement du genre : Homme
Locuteur : H13
========= Entraînement du GMM avec 8 composants ==========
Début de l'entraînement à : 15:45:13
Fin de l'entraînement à : 15:45:18
========= Entraînement du GMM avec 16 composants ==========
Début de l'entraînement à : 15:45:18
Fin de l'entraînement à : 15:45:38
========= Entraînement du GMM avec 32 composants ==========
Début de l'entraînement à : 15:45:38
Fin de l'entraînement à : 15:46:17
========= Entraînement du GMM avec 64 composants ==========
Début de l'entraînement à : 15:46:17
Fin de l'entraînement à : 15:47:13
========= Entraînement du GMM avec 128 composants ==========
Début de l'entraînement à : 15:47:13
Fin de l'entraînement à : 15:49:01
Locuteur : H14
========= Entraînement du GMM avec 8 composants ==========
Début de l'entraînement à : 15:49:08
Fin de l'entraînement à : 15:49:12
========= Entraînement du GMM avec 16 composants ==========
Début de l'e

- Cette fonction charge les caractéristiques MFCC pour les fichiers de test d'un locuteur spécifique.
- Elle parcourt les fichiers dans le dossier 'test' associé à un locuteur donné,
- charge les caractéristiques MFCC de chaque fichier, et les stocke dans une liste.

In [10]:
# Charge les caractéristiques MFCC pour les fichiers de test d'un locuteur spécifique.
def charger_caracteristiques_mfcc_test_par_locuteur(repertoire_mfcc, langue_cible, genre_cible, locuteur_cible):
    # Liste vide pour stocker les caractéristiques MFCC
    donnees_mfcc_test = []
    noms_segments = []  # Liste pour stocker les noms des segments

    # Chemin vers le dossier contenant les fichiers MFCC pour l'ensemble test
    dossier_test = os.path.join(repertoire_mfcc, langue_cible, genre_cible, locuteur_cible, 'test')

    # Parcours des fichiers dans le dossier de test
    for fichier_mfcc in os.listdir(dossier_test):
        # Chargement des caractéristiques MFCC à partir d'un fichier texte
        caracteristiques_mfcc = np.loadtxt(os.path.join(dossier_test, fichier_mfcc), delimiter=',')

        # Ajout des caractéristiques dans la liste des données de test
        donnees_mfcc_test.append(caracteristiques_mfcc)

        # Ajout du nom du fichier (segment) à la liste des segments
        noms_segments.append(fichier_mfcc)

    # Retourne les noms des segments et les données MFCC
    return noms_segments, donnees_mfcc_test

In [13]:
### N'exécutez pas cette cellule si les résultats sont déjà calculés et enregistrés dans Google Drive, car ce processus est long.

# Chemin vers le dossier contenant les modèles GMM pour les locuteurs
models_save_path = '/content/drive/My Drive/Reconnaissance/Models_GMM/R-Locuteur'
# Chemin pour stocker les résultats
results_save_path = '/content/drive/My Drive/Reconnaissance/Results_GMM/R-Locuteur'
# Chemin pour les données de test
MFCC_folder = '/content/drive/My Drive/Reconnaissance/Features_MFCC_Locuteur'
langues = os.listdir(MFCC_folder)
langues = ['English']

# Liste des nombres de composants GMM à tester
liste_nb_composants = [8, 16, 32, 64, 128]

# Évaluation pour chaque nombre de composants et chaque locuteur
for n_components in liste_nb_composants:
    print(f"Évaluation des modèles avec {n_components} composants.")

    # Parcours des langues
    for langue_cible in langues:
        # Parcours des genres (homme/femme)
        for genre_cible in os.listdir(os.path.join(MFCC_folder, langue_cible)):
            # Parcours des locuteurs
            for locuteur_cible in os.listdir(os.path.join(MFCC_folder, langue_cible, genre_cible)):
                print(f"Évaluation pour le locuteur : {locuteur_cible} ({langue_cible} - {genre_cible})")

                # Créer le dossier pour les résultats du locuteur si nécessaire
                locuteur_results_path = os.path.join(results_save_path, langue_cible, genre_cible, locuteur_cible)
                os.makedirs(locuteur_results_path, exist_ok=True)

                # Nom du fichier de sortie pour les résultats de test, incluant la langue et le genre
                modele_file_name = os.path.join(locuteur_results_path, f"Test_GMM_{n_components}_{langue_cible}_{genre_cible}_{locuteur_cible}.txt")

                # Ouverture du fichier pour écrire les résultats
                with open(modele_file_name, 'w') as f:
                    # Écriture de l'en-tête dans le fichier
                    entete = "Segment_Nom\tLocuteur_Cible\tScore_Max\tDecision_Model"
                    f.write(entete + "\n")

                    # Chargement des données de test pour le locuteur cible
                    nom_segs, test_data = charger_caracteristiques_mfcc_test_par_locuteur(MFCC_folder, langue_cible, genre_cible, locuteur_cible)

                    # Dictionnaire pour stocker les modèles GMM
                    locuteur_models = {}

                    # Chargement des modèles GMM pour chaque locuteur
                    for langue in langues:
                        for genre in os.listdir(os.path.join(MFCC_folder, langue)):
                            for locuteur in os.listdir(os.path.join(MFCC_folder, langue, genre)):
                                model_file_path = os.path.join(models_save_path, langue, genre, locuteur, f"GMM_{n_components}_{langue}_{genre}_{locuteur}_Locuteur.joblib")
                                locuteur_models[locuteur] = load(model_file_path)

                    # Itération sur chaque exemple de test
                    for i in range(len(test_data)):
                        # Extraction de l'exemple de test et de son nom de segment
                        test_example = test_data[i]
                        seg_name = nom_segs[i]

                        # Dictionnaire pour stocker les scores des modèles
                        scores = {}

                        # Évaluation des modèles GMM
                        for locuteur in locuteur_models.keys():
                            gmm = locuteur_models[locuteur]  # Modèle GMM pour le locuteur
                            score = gmm.score(test_example)  # Calcul du score du modèle
                            scores[locuteur] = score  # Ajout du score au dictionnaire

                        # Locuteur avec le score maximal
                        max_score_locuteur = max(scores, key=scores.get)
                        max_score = scores[max_score_locuteur]  # Score maximal

                        # Décision sur la correspondance avec le locuteur cible
                        decision = "oui" if max_score_locuteur == locuteur_cible else "non"

                        # Construction de la ligne de résultat et écriture dans le fichier
                        ligne = f"{seg_name}\t{locuteur_cible}\t{max_score}\t{decision}"
                        f.write(ligne + "\n")

                # print(f"Évaluation terminée pour {n_components} composants et le locuteur {locuteur_cible}. Résultats sauvegardés dans {modele_file_name}.")


Évaluation des modèles avec 8 composants.
Évaluation pour le locuteur : H13 (English - Homme)
Évaluation pour le locuteur : H14 (English - Homme)
Évaluation pour le locuteur : H12 (English - Homme)
Évaluation pour le locuteur : H15 (English - Homme)
Évaluation pour le locuteur : H16 (English - Homme)
Évaluation pour le locuteur : H17 (English - Homme)
Évaluation pour le locuteur : H18 (English - Homme)
Évaluation pour le locuteur : H19 (English - Homme)
Évaluation pour le locuteur : H2 (English - Homme)
Évaluation pour le locuteur : H3 (English - Homme)
Évaluation pour le locuteur : H4 (English - Homme)
Évaluation pour le locuteur : H6 (English - Homme)
Évaluation pour le locuteur : H5 (English - Homme)
Évaluation pour le locuteur : H7 (English - Homme)
Évaluation pour le locuteur : H8 (English - Homme)
Évaluation pour le locuteur : H9 (English - Homme)
Évaluation pour le locuteur : H1 (English - Homme)
Évaluation pour le locuteur : H11 (English - Homme)
Évaluation pour le locuteur : H

In [14]:
def afficher_precision_locuteur(resultats_path, liste_nb_composants):
    # Parcourir les langues dans le répertoire des résultats
    for langue_cible in os.listdir(resultats_path):
        # Parcourir les genres (homme/femme) dans chaque langue
        for genre_cible in os.listdir(os.path.join(resultats_path, langue_cible)):
            # Parcourir les locuteurs dans chaque genre
            for locuteur_cible in os.listdir(os.path.join(resultats_path, langue_cible, genre_cible)):
                print(f"Le locuteur {locuteur_cible} ({langue_cible} - {genre_cible}) :")

                # Pour chaque nombre de composants GMM
                for n_components in liste_nb_composants:
                    total_tests = 0
                    correct_predictions = 0

                    # Chemin vers les résultats pour chaque locuteur et nombre de composants
                    locuteur_results_path = os.path.join(resultats_path, langue_cible, genre_cible, locuteur_cible)

                    # Vérifier si le dossier existe
                    if os.path.exists(locuteur_results_path):
                        for file in os.listdir(locuteur_results_path):
                            if file.startswith(f"Test_GMM_{n_components}_{langue_cible}_{genre_cible}_{locuteur_cible}") and file.endswith(".txt"):  # les fichiers correspondant
                                file_path = os.path.join(locuteur_results_path, file)
                                with open(file_path, 'r') as f:
                                    # Sauter l'en-tête
                                    next(f)

                                    # Lire chaque ligne du fichier
                                    for line in f:
                                        # Extraire les informations de la ligne
                                        parts = line.strip().split("\t")
                                        seg_name, locuteur_cible_file, score_max, decision = parts

                                        # Comparer le locuteur cible avec le locuteur prédit
                                        if locuteur_cible_file == locuteur_cible:
                                            total_tests += 1
                                            if decision == "oui":  # Si la décision est "oui", c'est une bonne prédiction
                                                correct_predictions += 1

                        # Calculer et afficher la précision pour le modèle GMM avec n_components
                        if total_tests > 0:
                            precision = (correct_predictions / total_tests) * 100
                            print(f"   - La précision du modèle GMM {n_components} : {precision:.2f}%")
                        else:
                            print(f"   - Aucun test trouvé pour le modèle GMM {n_components}.")
                    else:
                        print(f"   - Aucun fichier trouvé pour le locuteur {locuteur_cible}.")

# Liste des composants GMM à tester
liste_nb_composants = [8, 16, 32, 64, 128]

# Appeler la fonction pour afficher les précisions
afficher_precision_locuteur('/content/drive/My Drive/Reconnaissance/Results_GMM/R-Locuteur', liste_nb_composants)

Le locuteur H1 (French - Homme) :
   - La précision du modèle GMM 8 : 0.00%
   - La précision du modèle GMM 16 : 0.00%
   - La précision du modèle GMM 32 : 0.00%
   - La précision du modèle GMM 64 : 0.00%
   - La précision du modèle GMM 128 : 0.00%
Le locuteur H2 (French - Homme) :
   - La précision du modèle GMM 8 : 0.00%
   - La précision du modèle GMM 16 : 0.00%
   - La précision du modèle GMM 32 : 0.00%
   - La précision du modèle GMM 64 : 0.00%
   - La précision du modèle GMM 128 : 0.00%
Le locuteur H3 (French - Homme) :
   - La précision du modèle GMM 8 : 0.00%
   - La précision du modèle GMM 16 : 0.00%
   - La précision du modèle GMM 32 : 0.00%
   - La précision du modèle GMM 64 : 0.00%
   - La précision du modèle GMM 128 : 0.00%
Le locuteur F1 (French - Femme) :
   - La précision du modèle GMM 8 : 0.00%
   - La précision du modèle GMM 16 : 0.00%
   - La précision du modèle GMM 32 : 0.00%
   - La précision du modèle GMM 64 : 0.00%
   - La précision du modèle GMM 128 : 0.00%
Le l

In [ ]:
# dossier_GMMs_Locuteur = '/content/drive/My Drive/Reconnaissance/Models_GMM/R-Locuteur'
# modeles_GMM['French']['Femme']['F1']